In [1]:
import os  
import sys
import pathlib

repo_path = pathlib.Path('.').absolute().parent
src_path = repo_path / 'src'

sys.path.insert(0, str(src_path)) 

In [ ]:
# synchronous testing for ro-crate upload
import fairscape_mds.models.rocrate as fairscape_rocrate
from fairscape_mds.config import get_fairscape_config
from uuid import uuid4

fairscapeConfig = get_fairscape_config()

minioClient = fairscapeConfig.CreateMinioClient()
mongoClient = fairscapeConfig.CreateMongoClient()

In [111]:
# create mongo collection information
mongoDB = mongoClient[fairscapeConfig.mongo.db]
identifierCollection = mongoDB[fairscapeConfig.mongo.identifier_collection]
rocrateCollection = mongoDB[fairscapeConfig.mongo.rocrate_collection]
userCollection = mongoDB[fairscapeConfig.mongo.user_collection]

In [ ]:
# import tasks for celery


### Load a dataverse RO-Crate

In [94]:
# load a zipped crate for usage for the test
zipped_crates = list(pathlib.Path('/Users/mal8ch-admin/Box Sync/cm4ai-data/Dataverse-Uploaded/').glob('*.zip'))
zipped_crates[0]

PosixPath('/Users/mal8ch-admin/Box Sync/cm4ai-data/Dataverse-Uploaded/4.cm4ai_chromatin_mda-mb-468_untreated_hierarchy_initialrun0.1alpha.zip')

In [ ]:

# uploading crate '4.cm4ai_chromatin_mda-mb-468_untreated_hierarchy_initialrun0.1alpha.zip'
zippedObjectFilename = zipped_crates[0].stem

In [16]:
transactionUUID = uuid4()

### Upload Zipped Crate to Minio

In [99]:
with zipped_crates[0].open('rb') as zipFileInstance:
    zipContent = zipFileInstance.read()

In [102]:
# must wrap byte content in a io.BytesIO for uploading
upload_output = fairscape_rocrate.UploadZippedCrate(
    minioClient,
    io.BytesIO(zipContent),
    fairscapeConfig.minio.rocrate_bucket,
    transactionUUID,
    zippedObjectFilename
)

DEBUG:urllib3.connectionpool:http://localhost:9000 "PUT /rocrate/5f501ede-82af-4623-9b10-c7e573c0840d/4.cm4ai_chromatin_mda-mb-468_untreated_hierarchy_initialrun0.1alpha HTTP/1.1" 200 0
INFO:rocrate:UploadZippedCrate	transaction=5f501ede-82af-4623-9b10-c7e573c0840d	message='Uploaded Zipped Crate Minio'	object_name='5f501ede-82af-4623-9b10-c7e573c0840d/4.cm4ai_chromatin_mda-mb-468_untreated_hierarchy_initialrun0.1alpha	' object_etag='90fe0fe2febade46e51439e9fc4d1b75'


In [32]:
str(upload_output[1])

'5f501ede-82af-4623-9b10-c7e573c0840d/4.cm4ai_chromatin_mda-mb-468_untreated_hierarchy_initialrun0.1alpha.zip'

In [36]:
transactionFolder = str(upload_output[1].parent)

In [33]:
import minio

## Minio Response

Returns a `urllib3.response.HTTPResponse` which can be read from for the unzip operation

In [82]:
try:
    response = minioClient.get_object(
            "rocrate",
            '5f501ede-82af-4623-9b10-c7e573c0840d/1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha.zip'
    )
    zippedContent = response.read()
    #extractStatus = fairscape_rocrate.UploadExtractedCrate(
    #    minioClient, 
    #    zippedContent, 
    #    response,
    #    fairscapeConfig.minio.default_bucket, 
    #    transactionFolder
    #)
finally:
    response.close()
    response.release_conn()
            

DEBUG:urllib3.connectionpool:Resetting dropped connection: localhost
DEBUG:urllib3.connectionpool:http://localhost:9000 "GET /rocrate/5f501ede-82af-4623-9b10-c7e573c0840d/1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha.zip HTTP/1.1" 200 32748


In [83]:
# check first 10 bytes
zippedContent[0:10]

b'PK\x03\x04\x14\x00\x00\x00\x00\x00'

In [68]:
import zipfile
import io

In [ ]:
# implementing main loop of upload extract crate

In [85]:
zipInstance = zipfile.ZipFile(io.BytesIO(zippedContent), "r")

In [87]:
zipInstance.infolist()[1::]

[<ZipInfo filename='1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha/error.log' compress_type=deflate filemode='-rw-r--r--' file_size=1237 compress_size=225>,
 <ZipInfo filename='1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha/output.log' compress_type=deflate filemode='-rw-r--r--' file_size=10584 compress_size=1897>,
 <ZipInfo filename='__MACOSX/1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha/._output.log' compress_type=deflate filemode='-rw-r--r--' file_size=176 compress_size=92>,
 <ZipInfo filename='1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha/ppi_gene_node_attributes.tsv' compress_type=deflate filemode='-rw-r--r--' file_size=55182 compress_size=12487>,
 <ZipInfo filename='__MACOSX/1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha/._ppi_gene_node_attributes.tsv' compress_type=deflate filemode='-rw-r--r--' file_size=233 compress_size=140>,
 <ZipInfo filename='1.cm4ai_chromatin_mda

In [88]:
# debugging upload zip
try:
    with zipfile.ZipFile(io.BytesIO(zippedContent), "r") as zip_file:
        for file_info in zip_file.infolist()[1::]:
            file_contents = zip_file.read(file_info.filename)

            # TODO the source filepath will not start at the root of the rocrate
            # but rather the full passed filename
            source_filepath = pathlib.Path(file_info.filename)

            # TODO file_info.filename can be 
            # Extracted/1.cm4ai_chromatin_mda-mb-468_untreated_imageloader_initialrun0.1alpha/ro-crate-metadata.json
            # this causes the GetROCrateMetadata function to fail later on becaues it looks only in
            # 1.cm4ai_chromatin_mda-mb-468_untreated_imageloader_initialrun0.1alpha/ 

            upload_filepath = pathlib.Path(transactionFolder) / source_filepath

            upload_result = minioClient.put_object(
                bucket_name= fairscapeConfig.minio.default_bucket, 
                object_name=str(upload_filepath), 
                data=io.BytesIO(file_contents), 
                length=len(file_contents)
            )
except Exception as e:
    print(e)


DEBUG:urllib3.connectionpool:Resetting dropped connection: localhost
DEBUG:urllib3.connectionpool:http://localhost:9000 "PUT /default/5f501ede-82af-4623-9b10-c7e573c0840d/1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha/error.log HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:http://localhost:9000 "PUT /default/5f501ede-82af-4623-9b10-c7e573c0840d/1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha/output.log HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:http://localhost:9000 "PUT /default/5f501ede-82af-4623-9b10-c7e573c0840d/__MACOSX/1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha/._output.log HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:http://localhost:9000 "PUT /default/5f501ede-82af-4623-9b10-c7e573c0840d/1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha/ppi_gene_node_attributes.tsv HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:http://localhost:9000 "PUT /default/5f501ede-82af-4623-9b10-c7e573c0840d/__MACOSX/

In [46]:
extractStatus = fairscape_rocrate.UploadExtractedCrate(
    minioClient, 
    zippedContent, 
    fairscapeConfig.minio.default_bucket, 
    transactionFolder
)

In [81]:
transactionFolder

'5f501ede-82af-4623-9b10-c7e573c0840d'

In [104]:
str(extractStatus[0])

'Success: True\tMessage: \tStatusCode: 200'

In [107]:
extractStatus[1]

['5f501ede-82af-4623-9b10-c7e573c0840d/3.cm4ai_chromatin_mda-mb-468_untreated_coembedfold1_initialrun0.1alpha',
 '5f501ede-82af-4623-9b10-c7e573c0840d/3.cm4ai_chromatin_mda-mb-468_untreated_coembedfold1_initialrun0.1alpha/muse_latent.txt',
 '5f501ede-82af-4623-9b10-c7e573c0840d/3.cm4ai_chromatin_mda-mb-468_untreated_coembedfold1_initialrun0.1alpha/muse_latent_x.txt',
 '5f501ede-82af-4623-9b10-c7e573c0840d/3.cm4ai_chromatin_mda-mb-468_untreated_coembedfold1_initialrun0.1alpha/muse_latent_y.txt',
 '5f501ede-82af-4623-9b10-c7e573c0840d/3.cm4ai_chromatin_mda-mb-468_untreated_coembedfold1_initialrun0.1alpha/error.log',
 '5f501ede-82af-4623-9b10-c7e573c0840d/3.cm4ai_chromatin_mda-mb-468_untreated_coembedfold1_initialrun0.1alpha/task_1693531560_start.json',
 '5f501ede-82af-4623-9b10-c7e573c0840d/3.cm4ai_chromatin_mda-mb-468_untreated_coembedfold1_initialrun0.1alpha/output.log',
 '5f501ede-82af-4623-9b10-c7e573c0840d/3.cm4ai_chromatin_mda-mb-468_untreated_coembedfold1_initialrun0.1alpha/task_1

In [103]:
# checking that minio has several unpacked objects
outputList = list(minioClient.list_objects('default', transactionFolder, recursive=True))

for obj in outputList:
    print(obj.object_name)

DEBUG:urllib3.connectionpool:Resetting dropped connection: localhost
DEBUG:urllib3.connectionpool:http://localhost:9000 "GET /default?delimiter=&encoding-type=url&list-type=2&max-keys=1000&prefix=5f501ede-82af-4623-9b10-c7e573c0840d HTTP/1.1" 200 0
5f501ede-82af-4623-9b10-c7e573c0840d/1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha/error.log
5f501ede-82af-4623-9b10-c7e573c0840d/1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha/output.log
5f501ede-82af-4623-9b10-c7e573c0840d/1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha/ppi_edgelist.tsv
5f501ede-82af-4623-9b10-c7e573c0840d/1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha/ppi_gene_node_attributes.errors
5f501ede-82af-4623-9b10-c7e573c0840d/1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha/ppi_gene_node_attributes.tsv
5f501ede-82af-4623-9b10-c7e573c0840d/1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha/ro-crate-metadat

### Get Metadata from ROCrate

In [ ]:


crateDistribution = fairscape_rocrate.ROCrateDistribution(
    extractedROCrateBucket= 'default',
    archivedROCrateBucket='rocrate',
    extractedObjectPath=extractStatus[1],
    archivedObjectPath=str(upload_output[1])
)

roCrateMetadata = fairscape_rocrate.GetMetadataFromCrate(
    minioClient,
    fairscapeConfig.minio.default_bucket,
    transactionFolder,
    # cratePath
    '1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha',
    # distribution
    crateDistribution
)

In [110]:
roCrateMetadata

{'@id': 'https://fairscape.net/ark:59852/rocrate-1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha',
 '@context': {'EVI': 'https://w3id.org/EVI#', '@vocab': 'https://schema.org/'},
 '@type': 'Dataset',
 'name': 'Initial integration run',
 'description': 'Ideker Lab CM4AI 0.1 alpha MDA-MB-468 untreated chromatin Initial integration run AP-MS Edgelist',
 'keywords': ['Ideker Lab',
  'CM4AI',
  '0.1 alpha',
  'MDA-MB-468',
  'untreated',
  'chromatin',
  'Initial integration run',
  'AP-MS edgelist download'],
 'isPartOf': [{'@id': 'ark:/Ideker_Lab',
   '@type': 'Organization',
   'name': 'Ideker Lab'},
  {'@id': 'ark:/Ideker_Lab/CM4AI', '@type': 'Project', 'name': 'CM4AI'}],
 '@graph': [{'@id': 'https://fairscape.net/ark:59852/software-cellmaps_ppidownloader-58sGTge',
   '@context': {'@vocab': 'https://schema.org/',
    'evi': 'https://w3id.org/EVI#'},
   'metadataType': 'https://w3id.org/EVI#Software',
   'url': 'https://github.com/idekerlab/cellmaps_ppidownloader',
 

## Publish PROV Metadata

In [112]:
fairscape_rocrate.PublishProvMetadata(roCrateMetadata, identifierCollection)

True

## Publish Identifier Metadata

In [113]:
fairscape_rocrate.PublishROCrateMetadata(roCrateMetadata, rocrateCollection)

True